In [ ]:
import pandas as pd
import numpy as np

In [ ]:
#df_abt = pd.read_csv("output/allcruises_df_validated_5with_zoomie.csv")
df_abt = pd.read_csv('output/update_allcruises_df_validated_5with_zoomie_20230303.csv')
df_abt =df_abt[df_abt["label"] != "Artefact"]
test = pd.read_csv("output/update_wo_artefacts_test_dataset_PS992_03032023.csv")

In [ ]:
df_abt_tv =df_abt[df_abt["object_cruise"] != "PS99.2"]
df_abt_tv_S = df_abt_tv[["label","count"]].groupby("label").sum()
df_abt_t_S = test[["label","count"]].groupby("label").sum()
df_abt_tv_S = df_abt_tv_S.rename(columns={"count": "train_val"})
df_abt_t_S =df_abt_t_S.rename(columns={"count": "test"})
df_plot = df_abt_tv_S.join(df_abt_t_S, on="label")
df_plot.to_csv("paper/tables/train_val_test.csv", sep=";")

In [ ]:
df_abt_99 =test[test["object_cruise"] == "PS99.2"]
df_abt_106 =df_abt[df_abt["object_cruise"] == "PS106.2"]
df_abt_107 =df_abt[df_abt["object_cruise"] == "PS107"]
df_abt_114 =df_abt[df_abt["object_cruise"] == "PS114"]
df_abt_122 =df_abt[df_abt["object_cruise"].isin(['PS122', 'PS122-2', 'PS122.1', 'PS122.2'])]

In [ ]:
all_cls = list(np.unique(df_abt["label"]))

In [ ]:
dl_99 = df_abt_99[["label","count"]].groupby("label").sum()
dl_106 = df_abt_106[["label","count"]].groupby("label").sum()
dl_107 = df_abt_107[["label","count"]].groupby("label").sum()
dl_114 = df_abt_114[["label","count"]].groupby("label").sum()
dl_122 = df_abt_122[["label","count"]].groupby("label").sum()

In [ ]:
for i in [dl_99, dl_106, dl_107, dl_114, dl_122]:
    print(len(i))
temp = dl_106.join(dl_99, on="label", rsuffix='_99')
temp = temp.join(dl_107, on="label", rsuffix='_107')
temp = temp.join(dl_114, on="label", rsuffix='_114')
temp = temp.join(dl_122, on="label",lsuffix='_106', rsuffix='_122')
temp['sum'] = temp[['count_106', 'count_99', 'count_107', 'count_114', 'count_122']].sum(axis=1)
temp.to_csv("paper/tables/statistics_over_cruises.csv", sep=";")

In [ ]:
from matplotlib import pyplot as plt
fig, axes = plt.subplots(nrows=5, ncols=1)
cruises = ["PS99.2", "PS106.2", "PS107", "PS114", "PS122"]
for index, df in enumerate([df_abt_99, df_abt_106, df_abt_107, df_abt_114, df_abt_122]):
    #print(df["object_cruise"].values[0])
    df = df[["label", "count"]]
    df=df.groupby("label").sum()
    sum1 = df.sum().item()
    df["count"] =df["count"]/sum1
    #print(df.sum().item())
    df.plot(ax =axes[index],  width=1.0, figsize=(25, 50), kind='bar')
    axes[index].set_ylabel('Number of Sample in % of total Samples')
    axes[index].set_title(f'{cruises[index]} #Samples={sum1}')
plt.tight_layout()
plt.savefig(f'paper/figures/CruiseStats.jpg')
    #print(df)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
a = df_abt_99[['label','count']].groupby('label').agg(sum)/df_abt_99[['label','count']].groupby('label').agg(sum).sum()
b = df_abt_106[['label','count']].groupby('label').agg(sum)/df_abt_106[['label','count']].groupby('label').agg(sum).sum()
c = df_abt_107[['label','count']].groupby('label').agg(sum)/df_abt_107[['label','count']].groupby('label').agg(sum).sum()
d = df_abt_114[['label','count']].groupby('label').agg(sum)/df_abt_114[['label','count']].groupby('label').agg(sum).sum()
e = df_abt_122[['label','count']].groupby('label').agg(sum)/df_abt_122[['label','count']].groupby('label').agg(sum).sum()

#t = cosine_similarity(X=d.values, Y=c.values, dense_output=True)
a_new = b.join(a, rsuffix='_PS99').drop('count', axis=1).fillna(0)
c_new = b.join(c, rsuffix='_PS107').drop('count', axis=1).fillna(0)
d_new = b.join(d, rsuffix='_PS114').drop('count', axis=1).fillna(0)
e_new = b.join(e, rsuffix='_PS122').drop('count', axis=1).fillna(0)
#t = cosine_similarity(X=a_new, Y=c_new, dense_output=True)
b_new =b
b_new = b_new.rename(columns={'count':'count_PS106'})
b_new = b_new.join(a_new)
b_new = b_new.join(c_new)
b_new = b_new.join(d_new)
b_new = b_new.join(e_new)
b_new.corr()

b_new['PS_train_val'] =  b_new['count_PS122'] + b_new['count_PS106'] + b_new['count_PS107'] +b_new['count_PS114']

#plt.imshow(b_new.corr())
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt

ax = sns.heatmap(b_new.corr(), linewidth=0.5, cmap='viridis')
#plt.show()
plt.tight_layout()
plt.savefig(f'paper/figures/HeatmapCorrelationLabelDistribution.jpg')
plt.show()